In [66]:
local_run = True
store_local = True
local_path = "/Users/algin/Greenwich/MSc Project/project_code/hate_speech_detection/data/uniform/"

In [5]:
try:
    import emoji
except ImportError:
    !pip install emoji
    import emoji

In [6]:
import pandas as pd
import numpy as np
import random
import string
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

if local_run:
    nltk.data.path.append('/Users/algin/VOLD/nltk_data')
    nltk.download('stopwords',download_dir='/Users/algin/VOLD/nltk_data')
    nltk.download('punkt',download_dir='/Users/algin/VOLD/nltk_data')
else:
    nltk.download('stopwords')
    nltk.download('punkt')
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/algin/VOLD/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/algin/VOLD/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/3-model-experiments/data/cleaned/cleaned_train_data.csv?raw=true")
test_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/3-model-experiments/data/cleaned/cleaned_test_data.csv?raw=true")

In [8]:
train_data.head()

,tweet_text,cyberbullying_type,is_cyberbullying,cleaned
0,"In other words #katandandre, your food was cra...",not_cyberbullying,0,word katandandr food crapilici
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying,0,aussietv white
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying,0,classi whore red velvet cupcak
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying,0,meh p thank head up concern anoth angri dude t...
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying,0,isi account pretend kurdish account like islam...


In [9]:
test_data.head()

,tweet_text,actual_value,is_cyberbullying,cleaned
0,"""\n\nI decided on """"breached"""" as a term to de...",[],0,decid breach term describ damag dam
1,Support Adding information regarding the 2007 ...,[],0,support ad inform regard 2007 easter denial se...
2,Stick to topic \n\nI would suggest cleaning up...,[],0,stick topic would suggest clean articl contain...
3,20 moments where e.g. or i.e. is used in an ar...,[],0,20 moment eg ie use articl print 40 page exces...
4,"""\n\n ADHD \n\nIt was a fight there for a whil...",[],0,adhd fight while thank scuro left seen month l...


In [10]:
malignant_raw_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/raw/malignant_comments/train.csv?raw=true")
unbalanced_train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/unbalanced/unbalanced_train_data.csv?raw=true")

In [11]:
len(malignant_raw_data)

159571

In [12]:
len(unbalanced_train_data)

47692

In [13]:
# Function to convert malignant data into a format to create new column 'is_cyberbullying'
def convert_malignant_tweets_format(data):
    new_data = pd.DataFrame()
    new_data['tweet_text'] = data['comment_text']
    new_data['actual_value'] = data.apply(lambda row: row[row == 1].index.tolist(), axis=1)
    new_data['is_cyberbullying'] = np.where(new_data['actual_value'].apply(lambda x:len(x)) == 0,0,1)
    return new_data

In [14]:
malignant_data = convert_malignant_tweets_format(malignant_raw_data)

In [15]:
len(malignant_data)

159571

In [16]:
unbalanced_train_data['is_cyberbullying'].value_counts()

1    39747
0     7945
Name: is_cyberbullying, dtype: int64

In [17]:
malignant_data['is_cyberbullying'].value_counts()

0    143346
1     16225
Name: is_cyberbullying, dtype: int64

**All bullying tweets can be added along with non bullying tweets**

In [21]:
columns = ['tweet_text','is_cyberbullying']

In [18]:
malignant_data['tweet_length'] = malignant_data['tweet_text'].str.len()
malignant_400_subset = malignant_data[malignant_data['tweet_length'] < 400]

In [19]:
malignant_400_subset['is_cyberbullying'].value_counts()

0    102298
1     13488
Name: is_cyberbullying, dtype: int64

In [30]:
unbalanced_train_data = unbalanced_train_data[columns]
malignant_400_subset = malignant_400_subset[columns]

**Create test dataset to make sure it has equal proportion of both datasets**

In [31]:
def create_test_data(data, size=4000):
    not_malignant_tweets = data[data['is_cyberbullying'] == 0]
    malignant_tweets = data[data['is_cyberbullying'] == 1]
    not_malignant_subset = not_malignant_tweets.sample(n=size//2, random_state=23)
    malignant_subset = malignant_tweets.sample(n=size//2, random_state=23)
    test_data = pd.concat([not_malignant_subset,malignant_subset])
    return test_data

In [32]:
test_data_A = create_test_data(unbalanced_train_data,size=5000)
test_data_B = create_test_data(malignant_400_subset,size=5000)

In [33]:
test_data_A['is_cyberbullying'].value_counts()

0    2500
1    2500
Name: is_cyberbullying, dtype: int64

In [34]:
test_data_B['is_cyberbullying'].value_counts()

0    2500
1    2500
Name: is_cyberbullying, dtype: int64

In [59]:
test_data = pd.concat([test_data_A,test_data_B])

**Balance train data by adding remaining tweets from B to A**

In [35]:
malignant_400_subset['is_cyberbullying'].value_counts()

0    102298
1     13488
Name: is_cyberbullying, dtype: int64

In [40]:
remaining_malignant = malignant_400_subset.drop(test_data_B.index)
remaining_malignant['is_cyberbullying'].value_counts()

0    99798
1    10988
Name: is_cyberbullying, dtype: int64

In [43]:
malignant_bullying_400_subset = remaining_malignant[remaining_malignant['is_cyberbullying']==1]
malignant_not_bullying_400_subset = remaining_malignant[remaining_malignant['is_cyberbullying']==0]

In [42]:
remaining_train_data = unbalanced_train_data.drop(test_data_A.index)
remaining_train_data['is_cyberbullying'].value_counts()

1    37247
0     5445
Name: is_cyberbullying, dtype: int64

In [45]:
new_train_data = pd.concat([remaining_train_data,malignant_bullying_400_subset])
new_train_data['is_cyberbullying'].value_counts()

1    48235
0     5445
Name: is_cyberbullying, dtype: int64

**Add 40K non bullying tweets to balance the train data**

In [46]:
additional_train_data_index = malignant_not_bullying_400_subset.sample(n=40000, random_state=23).index
transfer_malignant_data = malignant_not_bullying_400_subset.loc[additional_train_data_index]

In [55]:
uniform_train_data = pd.concat([new_train_data,transfer_malignant_data])

In [56]:
uniform_train_data['is_cyberbullying'].value_counts()

1    48235
0    45445
Name: is_cyberbullying, dtype: int64

In [60]:
test_data['is_cyberbullying'].value_counts()

0    5000
1    5000
Name: is_cyberbullying, dtype: int64

In [57]:
new_train_data['is_cyberbullying'].value_counts()

1    48235
0     5445
Name: is_cyberbullying, dtype: int64

In [50]:
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

In [51]:
banned_list= string.punctuation
punctuation_reg_exp = "[" + banned_list + "]"
emoji_reg_exp = get_emoji_regexp()

def stemmer(text):
    tokenized = nltk.word_tokenize(text)
    ps = PorterStemmer()
    return ' '.join([ps.stem(words) for words in tokenized])

def clean_text(text):
    text = text.replace('\r', '').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)

    text = [word for word in text.split() if word not in stop_words]
    text = ' '.join(text)

    text = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', text))
    
    text = re.sub(punctuation_reg_exp,"",text)

    text = re.sub("\s\s+" , " ", text)

    text = re.sub(emoji_reg_exp, r"", text)

    text = stemmer(text)
    return text

In [54]:
def clean_dataset(df):
    df['cleaned'] = df['tweet_text'].apply(clean_text)
    cleaned_df = df.drop_duplicates("cleaned")
    cleaned_df = cleaned_df[cleaned_df['cleaned']!='']
    return cleaned_df

In [61]:
cleaned_train_data = clean_dataset(uniform_train_data)
cleaned_train_data['cleaned'].duplicated().sum()

0

In [62]:
cleaned_test_data = clean_dataset(test_data)
cleaned_test_data['cleaned'].duplicated().sum()

0

In [63]:
cleaned_train_data['is_cyberbullying'].value_counts()

1    46137
0    45132
Name: is_cyberbullying, dtype: int64

In [64]:
cleaned_test_data['is_cyberbullying'].value_counts()

0    4928
1    4923
Name: is_cyberbullying, dtype: int64

In [68]:
if store_local:
    cleaned_train_data.to_csv(local_path + "uniform_train_data.csv",index=False)
    cleaned_test_data.to_csv(local_path + "uniform_test_data.csv",index=False)